In [8]:
import torch
import torch.nn.functional as F
import tokenizers
import transformers
from transformers import BertTokenizer, BertForMaskedLM
import os


In [19]:
def mask_all_tokens(sentence, tokenizer):
    masked_sentences = []
    tokenized_sentence = tokenizer(sentence, return_tensors='pt')
    for token_idx in range(1, len(tokenized_sentence['input_ids'][0]) - 1):
        masked_input = tokenizer(sentence, return_tensors='pt')  # tokenized_sentence.copy()
        masked_input['input_ids'][0][token_idx] = tokenizer.mask_token_id
        masked_sentences.append(masked_input)
    return masked_sentences

def compute_perplexity(sentence, tokenizer, model):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    masked_sentences = mask_all_tokens(sentence, tokenizer)
    original_tokens = tokenizer(sentence)['input_ids']
    sent_results = {'most_prob': [], 'correct_prob': [], 'top_1': [], 'top_5': [], 'top_10': []}
    for sent_idx, masked_sentence in enumerate(masked_sentences):
        masked_idx = sent_idx + 1
        correct_token = original_tokens[masked_idx]
        with torch.no_grad():
            outputs = model(**masked_sentence.to(device))
            logits = outputs.logits[0, masked_idx]
            probs = logits.softmax(dim=-1)
            values, predictions = probs.topk(10)
            # sent_results['top_1'].append(1 if correct_token in predictions[0] else 0)
            # sent_results['top_5'].append(1 if correct_token in predictions[:5] else 0)
            # sent_results['top_10'].append(1 if correct_token in predictions else 0)
            sent_results['correct_prob'].append(probs[correct_token].item())
            # sent_results['most_prob'].append(values[0].item())
    final_result = sum(sent_results['correct_prob']) / len(sent_results['correct_prob'])
    return final_result


In [22]:
tokenizer = BertTokenizer.from_pretrained("dbmdz/bert-base-italian-cased")
model = BertForMaskedLM.from_pretrained("C:/Users/bergo/OneDrive - University of Pisa/Tesi Magistrale/models/ANTI_CURRICULUM/final_pretrained_model")
frase = "Nonostante l'apparente ineluttabilità del destino, intessuto com'è nel tessuto stesso del continuum spazio-temporale, la teoria della relatività generale di Einstein suggerisce che le curvature dello spazio-tempo, influenzate dalla distribuzione della massa-energia, possono dare origine a fenomeni astrofisici tanto enigmatici quanto i buchi neri e le onde gravitazionali."
result = compute_perplexity(frase, tokenizer, model)

In [23]:
result

0.1436407217211672